# Recomendador de Ingredientes para Nuevas Recetas

Trabajo Fin de Máster

Presentado por: Bayas Fernández, Jaime Mauricio

Director: Baldominos Gómez, Alejandro

In [398]:
#LIBRERIAS Y CARGA DATASET

#Librerias utilizadas
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

#Importamos la información del data set "IndianFoodDatasetXLS_v1.0.xlsx" desde Github
#URL del dataset
url="https://github.com/mbayas13/tfm_recomendador_ingredientes/raw/main/IndianFoodDatasetXLS_v2.1.xlsx"

#Carga de la información
data_recipe=pd.read_excel(url)

#impresion
data_recipe

,Id,RecipeName,Ingredients,Servings,Diet
0,1,Masala Karela Recipe,"6 Karela (Bitter Gourd/ Pavakkai) - deseeded,S...",6,Diabetic Friendly
1,2,Spicy Tomato Rice (Recipe),"2-1 / 2 cups rice - cooked, 3 tomatoes, 3 teas...",3,Vegetarian
2,3,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1-1/2 cups Rice Vermicelli Noodles (Thin),1 On...",4,High Protein Vegetarian
3,4,Gongura Chicken Curry Recipe - Andhra Style Go...,"500 grams Chicken,2 Onion - chopped,1 Tomato -...",4,Non Vegeterian
4,5,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"1 tablespoon chana dal, 1 tablespoon white ura...",4,Vegetarian
...,...,...,...,...,...
6292,14030,Saffron Paneer Peda Recipe,2 cups Paneer (Homemade Cottage Cheese) - crum...,4,Vegetarian
6293,14038,Italian Arancini Rice Balls Recipe With Delici...,1-1/2 cup Risotto - cooked risotto (recipe bel...,3,Vegetarian
6294,14045,Quinoa Phirnee Recipe (Quinoa Milk Pudding),"1 cup Quinoa,3/4 cup Sugar,1 teaspoon Cardamom...",2,Vegetarian
6295,14165,Ullikadala Pulusu Recipe | Spring Onion Curry,150 grams Spring Onion (Bulb & Greens) - chopp...,2,Vegetarian


In [399]:
#FUNCIONES UTILITARIAS

#Función para crear una nueva receta
# Entrada:
#    recipe_name: Nombre de la receta
#    servings: Número de porciones
#    diet: Tipo de dieta
#    data_recipe: Dataset con la información de las recetas disponibles
# Retorna: Dataframe con la nueva receta inluida
def crear_receta (recipe_name,servings,diet,data_recipe):
    #Generamos el Id de la nueva receta
    nuevo_id=1
    nuevo_id+=data_recipe["Id"].max()
    nueva_receta={"Id":nuevo_id,"RecipeName":recipe_name,"Ingredients":" ","Servings":servings,"Diet":diet}
    data_recipe = data_recipe.append(nueva_receta, ignore_index=True)
    return data_recipe

#Función para crear un nuevo ingrediente
# Entrada:
#    ingredient_name: Descripción del nuevo ingrediente
#    recipe_name: Nombre de la receta
#    servings: Número de porciones
#    diet: tipo de dieta
#    data_ingredient: Dataframe con la información de los ingredientes por receta
# Retorna: Dataframe con el nuevo ingrediente inluido
def crear_ingrediente (ingredient_name,recipe_name,servings,diet,data_ingredient):
    #Generamos el Id de la nueva receta
    nuevo_id=1
    nuevo_id+=data_ingredient["Id"].max()
    nuevo_ingrediente={"Id":nuevo_id,"RecipeName":recipe_name,"Ingredients":ingredient_name,
                       "Servings":servings,"Diet":diet,"metaData":""}
    data_ingredient = data_ingredient.append(nuevo_ingrediente, ignore_index=True)
    return data_ingredient

#Funcion que elimina espacios en blanco de una cadena
# Entrada:
#    x: cadena
# Retorna: Cadena sin espacios en blanco, caso contrario retorna una cadena vacia
def eliminar_espacios(x):
    if isinstance(x,str):
        return str.lower(x.replace(" ",""))
    else:
        return ""

#Funcion que coloca en minúsculas una cadena
# Entrada:
#    x: cadena
# Retorna: Cadena en minúsculas, caso contrario retorna una cadena vacia
def minusculas(x):
    if isinstance(x,str):
        return str.lower(x)
    else:
        return ""

#Funcion que combina la  información de nombre de la receta (recipe_name)
#porciones (servings) y tipo de dieta (diet)
# Entrada:
#    df: dataframe con la información de recetas
# Retorna: String con la información de entrada
def combinar_metadata(df):
    return df['RecipeName'] + ' ' + str(df['Servings']) + ' ' + df['Diet']

In [400]:
#LÓGICA DE LA SOLUCIÓN

#PASO 1: Definimos La nueva receta a crear
recipe_name="Mexican Style Black Bean Burrito Recipe"
servings=4
diet="Vegetarian"


In [401]:
#PASO 2: Incluimos la nueva receta en el dataset
data_recipe=crear_receta(recipe_name,servings,diet,data_recipe)

#impresion
data_recipe



C:\Users\DELL\AppData\Local\Temp/ipykernel_2308/4146054393.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_recipe = data_recipe.append(nueva_receta, ignore_index=True)


,Id,RecipeName,Ingredients,Servings,Diet
0,1,Masala Karela Recipe,"6 Karela (Bitter Gourd/ Pavakkai) - deseeded,S...",6,Diabetic Friendly
1,2,Spicy Tomato Rice (Recipe),"2-1 / 2 cups rice - cooked, 3 tomatoes, 3 teas...",3,Vegetarian
2,3,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1-1/2 cups Rice Vermicelli Noodles (Thin),1 On...",4,High Protein Vegetarian
3,4,Gongura Chicken Curry Recipe - Andhra Style Go...,"500 grams Chicken,2 Onion - chopped,1 Tomato -...",4,Non Vegeterian
4,5,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"1 tablespoon chana dal, 1 tablespoon white ura...",4,Vegetarian
...,...,...,...,...,...
6293,14038,Italian Arancini Rice Balls Recipe With Delici...,1-1/2 cup Risotto - cooked risotto (recipe bel...,3,Vegetarian
6294,14045,Quinoa Phirnee Recipe (Quinoa Milk Pudding),"1 cup Quinoa,3/4 cup Sugar,1 teaspoon Cardamom...",2,Vegetarian
6295,14165,Ullikadala Pulusu Recipe | Spring Onion Curry,150 grams Spring Onion (Bulb & Greens) - chopp...,2,Vegetarian
6296,14167,Kashmiri Style Kokur Yakhni Recipe-Chicken Coo...,"1 kg Chicken - medium pieces,1/2 cup Mustard o...",4,Non Vegeterian


In [402]:
#PASO 3: Ejecutamos las siguientes tareas sobre el dataframe de recetas:
# - Transformamos a minúsculas la información de las variables "RecipeName","Ingredients","Diet"
# - Eliminamos los espacios en blanco de la información de la variable "Diet"
for variable in ["RecipeName","Ingredients","Diet"]:
    if variable=="Diet":
        data_recipe[variable] = data_recipe[variable].apply(eliminar_espacios)
    else:
        data_recipe[variable] = data_recipe[variable].apply(minusculas)

#Colocamos en minusculas el contenido de la variable "recipe_name" y "diet"
recipe_name=recipe_name.lower()
diet=diet.lower()
        
#impresion
data_recipe

,Id,RecipeName,Ingredients,Servings,Diet
0,1,masala karela recipe,"6 karela (bitter gourd/ pavakkai) - deseeded,s...",6,diabeticfriendly
1,2,spicy tomato rice (recipe),"2-1 / 2 cups rice - cooked, 3 tomatoes, 3 teas...",3,vegetarian
2,3,ragi semiya upma recipe - ragi millet vermicel...,"1-1/2 cups rice vermicelli noodles (thin),1 on...",4,highproteinvegetarian
3,4,gongura chicken curry recipe - andhra style go...,"500 grams chicken,2 onion - chopped,1 tomato -...",4,nonvegeterian
4,5,andhra style alam pachadi recipe - adrak chutn...,"1 tablespoon chana dal, 1 tablespoon white ura...",4,vegetarian
...,...,...,...,...,...
6293,14038,italian arancini rice balls recipe with delici...,1-1/2 cup risotto - cooked risotto (recipe bel...,3,vegetarian
6294,14045,quinoa phirnee recipe (quinoa milk pudding),"1 cup quinoa,3/4 cup sugar,1 teaspoon cardamom...",2,vegetarian
6295,14165,ullikadala pulusu recipe | spring onion curry,150 grams spring onion (bulb & greens) - chopp...,2,vegetarian
6296,14167,kashmiri style kokur yakhni recipe-chicken coo...,"1 kg chicken - medium pieces,1/2 cup mustard o...",4,nonvegeterian


In [403]:
#PASO 4: Agregamos un campo al dataframe de recetas con
# la información combinada de las variables "RecipeName","Servings","Diet"
data_recipe['metaData'] = data_recipe.apply(combinar_metadata, axis=1)

#impresion
data_recipe

,Id,RecipeName,Ingredients,Servings,Diet,metaData
0,1,masala karela recipe,"6 karela (bitter gourd/ pavakkai) - deseeded,s...",6,diabeticfriendly,masala karela recipe 6 diabeticfriendly
1,2,spicy tomato rice (recipe),"2-1 / 2 cups rice - cooked, 3 tomatoes, 3 teas...",3,vegetarian,spicy tomato rice (recipe) 3 vegetarian
2,3,ragi semiya upma recipe - ragi millet vermicel...,"1-1/2 cups rice vermicelli noodles (thin),1 on...",4,highproteinvegetarian,ragi semiya upma recipe - ragi millet vermicel...
3,4,gongura chicken curry recipe - andhra style go...,"500 grams chicken,2 onion - chopped,1 tomato -...",4,nonvegeterian,gongura chicken curry recipe - andhra style go...
4,5,andhra style alam pachadi recipe - adrak chutn...,"1 tablespoon chana dal, 1 tablespoon white ura...",4,vegetarian,andhra style alam pachadi recipe - adrak chutn...
...,...,...,...,...,...,...
6293,14038,italian arancini rice balls recipe with delici...,1-1/2 cup risotto - cooked risotto (recipe bel...,3,vegetarian,italian arancini rice balls recipe with delici...
6294,14045,quinoa phirnee recipe (quinoa milk pudding),"1 cup quinoa,3/4 cup sugar,1 teaspoon cardamom...",2,vegetarian,quinoa phirnee recipe (quinoa milk pudding) 2 ...
6295,14165,ullikadala pulusu recipe | spring onion curry,150 grams spring onion (bulb & greens) - chopp...,2,vegetarian,ullikadala pulusu recipe | spring onion curry ...
6296,14167,kashmiri style kokur yakhni recipe-chicken coo...,"1 kg chicken - medium pieces,1/2 cup mustard o...",4,nonvegeterian,kashmiri style kokur yakhni recipe-chicken coo...


In [404]:
#PASO 5: Definimos la matriz TF_IDF para la variable "metaData"
#Definimos el modelo con base a la libreria sklearn
tfidf= TfidfVectorizer(stop_words="english")

#Se reemplazan las cadenas vacias de la variable "metaData" con NAN
data_recipe["metaData"]=data_recipe["metaData"].fillna("")

#Generamos la matriz TF-IDF 
tfidf_matrix=tfidf.fit_transform(data_recipe["metaData"])

#Imprimimos el tamaño de la matriz generada
tfidf_matrix.shape



(6298, 3926)

In [405]:
#PASO 6: Generamos la matriz de similitud de coseno
cosine_sim_recipe=linear_kernel(tfidf_matrix,tfidf_matrix)

#impresion
cosine_sim_recipe

array([[1.        , 0.01512505, 0.00598763, ..., 0.00776323, 0.00685248,
        0.00889129],
       [0.01512505, 1.        , 0.0071569 , ..., 0.02654112, 0.00819065,
        0.03039778],
       [0.00598763, 0.0071569 , 1.        , ..., 0.00367342, 0.00324247,
        0.0042072 ],
       ...,
       [0.00776323, 0.02654112, 0.00367342, ..., 1.        , 0.00420401,
        0.01560226],
       [0.00685248, 0.00819065, 0.00324247, ..., 0.00420401, 1.        ,
        0.04667156],
       [0.00889129, 0.03039778, 0.0042072 , ..., 0.01560226, 0.04667156,
        1.        ]])

In [406]:
#PASO 7: Generamos una serie donde el índice es el nombre de la receta
# y el valor es el índice de la misma 
indices_recipe = pd.Series(data_recipe.index, index=data_recipe['RecipeName']).drop_duplicates()

indices_recipe



RecipeName
masala karela recipe                                                        0
spicy tomato rice (recipe)                                                  1
ragi semiya upma recipe - ragi millet vermicelli breakfast                  2
gongura chicken curry recipe - andhra style gongura chicken                 3
andhra style alam pachadi recipe - adrak chutney (recipe)                   4
                                                                         ... 
italian arancini rice balls recipe with delicious pasta dipping sauce    6293
quinoa phirnee recipe (quinoa milk pudding)                              6294
ullikadala pulusu recipe | spring onion curry                            6295
kashmiri style kokur yakhni recipe-chicken cooked in yogurt              6296
mexican style black bean burrito recipe                                  6297
Length: 6298, dtype: int64

In [407]:
#PASO 8: Creamos la función recomendadora de recetas
def recipe_recommender(recipe_name,cosine_sim_recipe=cosine_sim_recipe,data_recipe=data_recipe,indices_recipe=indices_recipe):
    #Obtenemos el indice de la receta asociado al nombre de la receta
    idx=indices_recipe[recipe_name]
    
    #Obtener los valores de similutud entre la nueva receta y todas las recetas disponibles
    sim_scores=list(enumerate(cosine_sim_recipe[idx]))
    
    #Se ordena de manera descendente la lista con base al valor de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    #Obtenemos las 10 recetas con los valores de similitud mál altos,
    #se excluye el primer valor ya que corresponde a la similiud entre la 
    #misma receta (valor igual a 1)
    sim_scores = sim_scores[1:11]
    
    #Obtenemos los índices de las 10 recetas con los valores
    #de similitud más altos
    recipe_indices = [i[0] for i in sim_scores]
    
    #Retornamos el dataframe con las 10 recetas más similares
    #a la nueva receta creada
    #return data_recipe['RecipeName'].iloc[movie_indices]
    return data_recipe.iloc[recipe_indices]

In [408]:
#PASO 9: Obtener el dataframe con las 10 recetas recomendadas con base
# a la nueva receta creada
recipe_recomendadas=recipe_recommender(recipe_name)

#impresion
recipe_recomendadas

,Id,RecipeName,Ingredients,Servings,Diet,metaData
698,786,mexican black bean corn soup recipe,"1 onion - finely chopped,2 cloves garlic - fin...",4,vegetarian,mexican black bean corn soup recipe 4 vegetarian
231,248,mexican vegetarian burrito bowl recipe,"2 cups cooked rice - fluffy,1 coriander (dhani...",4,vegetarian,mexican vegetarian burrito bowl recipe 4 veget...
2849,3760,healthy black bean stuffed burrito recipe with...,"3/4 cup mixed grains - (amaranth,1 1/2 cup wat...",4,vegetarian,healthy black bean stuffed burrito recipe with...
3447,4782,black bean noodles recipe,"3 cups hakka noodles - boiled,10 green beans (...",2,vegetarian,black bean noodles recipe 2 vegetarian
926,1086,mexican style vegetarian chimichanga recipe-de...,"1 carrot (gajjar) - peeled and chopped small,1...",4,vegetarian,mexican style vegetarian chimichanga recipe-de...
5480,9366,mexican quinoa bean salad recipe,"1 cup black beans - cooked/canned,1/2 cup quin...",4,vegetarian,mexican quinoa bean salad recipe 4 vegetarian
1213,1449,chicken burrito recipe,"300 grams chicken - cut into stripes,1 onion -...",4,nonvegeterian,chicken burrito recipe 4 nonvegeterian
2852,3764,open bean and veggies burrito muffin recipe,"3 whole wheat flour tortilla - cut in half,1 c...",3,vegetarian,open bean and veggies burrito muffin recipe 3 ...
2736,3572,red bean mexican burrito bowl recipe - roz ka ...,"1 cup rajma (large kidney beans) - cooked,4 cl...",4,vegetarian,red bean mexican burrito bowl recipe - roz ka ...
6098,12119,homemade black bean burger recipe,"1 cup canned black beans,2 tablespoons extra v...",4,vegetarian,homemade black bean burger recipe 4 vegetarian


In [409]:
#PASO 10: Generar un dataframe con los ingredientes de las recetas recomendadas
#Agregamos una fila por cada ingrediente de cada receta
data_ingredient=recipe_recomendadas.assign(Ingredients=recipe_recomendadas["Ingredients"]
                                                   .str.split(",")).explode('Ingredients',ignore_index=True)

#impresion
data_ingredient

,Id,RecipeName,Ingredients,Servings,Diet,metaData
0,786,mexican black bean corn soup recipe,1 onion - finely chopped,4,vegetarian,mexican black bean corn soup recipe 4 vegetarian
1,786,mexican black bean corn soup recipe,2 cloves garlic - finely chopped,4,vegetarian,mexican black bean corn soup recipe 4 vegetarian
2,786,mexican black bean corn soup recipe,2 cups black beans - cooked/canned (reserve th...,4,vegetarian,mexican black bean corn soup recipe 4 vegetarian
3,786,mexican black bean corn soup recipe,1/3 cup homemade tomato puree,4,vegetarian,mexican black bean corn soup recipe 4 vegetarian
4,786,mexican black bean corn soup recipe,1/2 cup sweet corn,4,vegetarian,mexican black bean corn soup recipe 4 vegetarian
...,...,...,...,...,...,...
200,12119,homemade black bean burger recipe,2 cloves garlic - grated,4,vegetarian,homemade black bean burger recipe 4 vegetarian
201,12119,homemade black bean burger recipe,1 inch ginger - grated,4,vegetarian,homemade black bean burger recipe 4 vegetarian
202,12119,homemade black bean burger recipe,mint leaves (pudina) - a small bunch finely ch...,4,vegetarian,homemade black bean burger recipe 4 vegetarian
203,12119,homemade black bean burger recipe,1/2 cup whole wheat bread crumbs,4,vegetarian,homemade black bean burger recipe 4 vegetarian


In [410]:
#PASO 11: Definimos el ingrediente propuesto para la nueva receta
ingredient_name="Garlic"

#Colocamos en minusculas el valor del ingrediente
ingredient_name=ingredient_name.lower()


In [411]:
#PASO 12: Incluimos el ingrediente en el dataframe de recetas recomendadas
data_ingredient=crear_ingrediente(ingredient_name,recipe_name,servings,diet,data_ingredient)

#impresion
data_ingredient

C:\Users\DELL\AppData\Local\Temp/ipykernel_2308/4146054393.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_ingredient = data_ingredient.append(nuevo_ingrediente, ignore_index=True)


,Id,RecipeName,Ingredients,Servings,Diet,metaData
0,786,mexican black bean corn soup recipe,1 onion - finely chopped,4,vegetarian,mexican black bean corn soup recipe 4 vegetarian
1,786,mexican black bean corn soup recipe,2 cloves garlic - finely chopped,4,vegetarian,mexican black bean corn soup recipe 4 vegetarian
2,786,mexican black bean corn soup recipe,2 cups black beans - cooked/canned (reserve th...,4,vegetarian,mexican black bean corn soup recipe 4 vegetarian
3,786,mexican black bean corn soup recipe,1/3 cup homemade tomato puree,4,vegetarian,mexican black bean corn soup recipe 4 vegetarian
4,786,mexican black bean corn soup recipe,1/2 cup sweet corn,4,vegetarian,mexican black bean corn soup recipe 4 vegetarian
...,...,...,...,...,...,...
201,12119,homemade black bean burger recipe,1 inch ginger - grated,4,vegetarian,homemade black bean burger recipe 4 vegetarian
202,12119,homemade black bean burger recipe,mint leaves (pudina) - a small bunch finely ch...,4,vegetarian,homemade black bean burger recipe 4 vegetarian
203,12119,homemade black bean burger recipe,1/2 cup whole wheat bread crumbs,4,vegetarian,homemade black bean burger recipe 4 vegetarian
204,12119,homemade black bean burger recipe,1/2 cup oats flour,4,vegetarian,homemade black bean burger recipe 4 vegetarian


In [412]:
#PASO 13: Definimos la matriz TF_IDF para la variable "Ingredients"
#Definimos el modelo con base a la libreria sklearn
tfidf_ing= TfidfVectorizer(stop_words="english")

#Se reemplazan las cadenas vacias de la variable "Ingredients" con NAN
data_ingredient["Ingredients"]=data_ingredient["Ingredients"].fillna("")

#Generamos la matriz TF-IDF 
tfidf_matrix_ing=tfidf_ing.fit_transform(data_ingredient["Ingredients"])

#Imprimimos el tamaño de la matriz generada
tfidf_matrix_ing.shape

(206, 179)

In [413]:
#PASO 14: Generamos la matriz de similitud de coseno
cosine_sim_ingredient=linear_kernel(tfidf_matrix_ing,tfidf_matrix_ing)

#impresion
cosine_sim_ingredient

array([[1.        , 0.4625161 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.4625161 , 1.        , 0.        , ..., 0.        , 0.        ,
        0.54261021],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.09013337,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.09013337, 1.        ,
        0.        ],
       [0.        , 0.54261021, 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [414]:
#PASO 15: Generamos una serie donde el índice es el nombre del ingrediente
# y el valor es el índice de la misma 
indices_ingredient = pd.Series(data_ingredient.index, index=data_ingredient['Ingredients']).drop_duplicates()

#Impresion
indices_ingredient

Ingredients
1 onion - finely chopped                                     0
2 cloves garlic - finely chopped                             1
2 cups black beans - cooked/canned (reserve the liquid)      2
1/3 cup homemade tomato puree                                3
1/2 cup sweet corn                                           4
                                                          ... 
1 inch ginger - grated                                     201
mint leaves (pudina) - a small bunch finely chopped        202
1/2 cup whole wheat bread crumbs                           203
1/2 cup oats flour                                         204
garlic                                                     205
Length: 206, dtype: int64

In [415]:
#PASO 16: Creamos la función recomendadora de ingredientes
def ingredient_recommender(ingredient_name,cosine_sim_ingredient=cosine_sim_ingredient,
                           data_ingredient=data_ingredient,indices_ingredient=indices_ingredient):
    #Obtenemos el indice asociado al nombre del ingrediente
    idx_ing=indices_ingredient[ingredient_name]
    
    #Obtener los valores de similutud entre el nuevo ingrediente y todas los ingredientes disponibles
    sim_scores_ing=list(enumerate(cosine_sim_ingredient[idx_ing]))
    
    #Se ordena de manera descendente la lista con base al valor de similitud
    sim_scores_ing = sorted(sim_scores_ing, key=lambda x: x[1], reverse=True)
    
    #Obtenemos los 3 ingredientes con los valores de similitud mál altos,
    #se excluye el primer valor ya que corresponde a la similiud entre el 
    #mismo ingrediente (valor igual a 1)
    sim_scores_ing = sim_scores_ing[1:4]
    
    #Obtenemos los índices de los 3 ingredientes con los valores
    #de similitud más altos
    ingredient_indices_result = [i[0] for i in sim_scores_ing]
    
    #Retornamos los 3 ingredientes recomendados
    return data_ingredient['Ingredients'].iloc[ingredient_indices_result]
    

In [416]:
#PASO 17: Obtener el resultado con los 3 ingredientes recomendadas con base
# al nuevo ingrediente creado
ingredient_recommender(ingredient_name)


173              3 cloves garlic
81     4 cloves garlic - chopped
117    4 cloves garlic - chopped
Name: Ingredients, dtype: object